# Hands On ML Chapter 11 - Training Deep Neural Nets

In [1]:
import tensorflow as tf
from functools import partial

## Vanishing/Exploding Gradients Problems

By default the tf.layers.dense() uses Xavier initialization, you can change it to He initialization by:

In [2]:
he_init = tf.contrib.layers.variance_scaling_initializer()
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, kernel_initializer=he_init, name='hidden1')

Elu:

In [3]:
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.elu, name="hidden1")

Leaky Relu:

In [4]:
def leaky_relu(z, name=None):
    return tf.maximum(0.01 *z, z, name=name)

hidden1 = tf.layers.dense(X, n_hidden1, activation=leaky_relu, name="hidden1")

### Batch Normalization

Batch Normalization guarantee that vanishing/exploding gradient problems wont come back during training.

In [5]:
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
training = tf.placeholder_with_default(False, shape=(), name="training") # True or False value, True for training, False for testing
#^true/false value say if we want to use mean and st dev from mini batch (training) or whole set (testing)

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1") # fully connected layers
bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
bn1_act = tf.nn.elu(bn1)
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = tf.layers.batch_normalization(logits_before_bn, training=training, momentum=0.9)

To avoid repetitive code you can use partial from standard python tools:

In [ ]:
my_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name='hidden1')
bn1 = my_batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1)
hidden2 = tf.layers.dense(hidden1, n_hidden2, name='hidden2')
bn2 = my_batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)
logits_before_bn = tf.layers.dense(hidden2, n_outputs, name='outputs')
logits = my_batch_norm_layer(logits_before_bn)

Execution phase: (for more see previous chapter about GD with ANN)

In [ ]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op, extra_update_ops],
                    feed_dict={training: True, X: X_batch, y: y_batch})
            accuracy_val = accuracy.eval(feed_dict = {X: mnist.test.images,
                                                      y: mnist.test.labels})
            print(epoch, "Test accuracy:", accuracy_val)
            
        save_path = saver.save(sess, "./my_model_final.ckpt")

### Gradient Clipping

Not that populat as Batch Normalization but it is worth to be known. It compute gradients, then clip them betwen -threshold and threshold.

In [ ]:
threshold = 1.0
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)
capped_gvs = [(tf.clip_by_value(grad, -threshold, threshold), var)
              for grad, var in grads_and_vars]
training_op = optimizer.apply_gradients(capped_gvs)

## Reusing Pretrained Layers

It is not good idea to train a very large DNN from scratch. Better idea is to try to find existing model and use part of it to your task. Reuse the lower layers of this network.

### Reusing a TensorFlow Model

You can restore your old tensorflow model followoing these steps:

In [3]:
saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta") #import operations into the default graph

#get a handle on the tensors and operations you need for training
X = tf.get_default_graph().get_tensor_by_name("X:0") # X is name, 0 is the first output, 1 is second output, 2 third output...
y = tf.get_default_graph().get_tensor_by_name("y:0")
accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")
training_op = tf.get_default_graph().get_operation_by_name("GradientDescent")

#if you dont know names of operations you can chect it by TensorBoard or by:
for op in tf.get_default_graph().get_operations():
    print(op.name)
    
#it you are the author of the original model, you can give models clear names or create good documentation
#you can also create collection containing all important operations:
for op in (X, y, accuracy, training_op):
    tf.add_to_collection("my_important_ops", op)
#to reuse it people simply can write:
X, y, accuracy, training_op = tf.get_collection("my_important_ops")

#in general you will need to reuse just a part of the original model, typically the lower layers
#the code below restores only hidden layers 1,2 and 3

#[....] build the new model with the same hidden layers 1-3 as before
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="hidden[123]") #regular expression
reuse_vars_dict = dict([(var.op.name, var) for var in reuse_vars])
restore_saver = tf.train.Saver(reuse_vars_dict) #to restore layers 1-3

init = tf.global_variables_initializer() #to init all variables, old and new
saver = tf.train.Saver() #to save the new model

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")
    #[....] train the model
    save_path = saver.save(sess, "./my_new_model_final.ckpt")

The more similar the tasks are the more layers you can reuse. Sometimes you can only change the output layer.

### Reusing Models from Other Frameworks

If the model was trained with another python framework (for example Theano), you will need to load the model parameters manually. The code below is is example how to copy the weight and biases from the first hidden layer of model trained using another framework.

In [ ]:
original_w = #[...] load weights from other framework
original_b = #[...] load biases from other framework

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name='hidden1')
#[...] build the rest of the model

graph = tf.get_default_graph()
assign_kernel = graph.get_operation_by_name("hidden1/kernel/Assign")
assing_bias = graph.get_operation_by_name("hidden1/bias/Assign")
init_kernel = assign_kernel.inputs[1]
init_bias = assign_bias.inputs[1]

init = tf.global_variables_initializer()

with tf.Session as sess:
    sess.run(init, feed_dict={init_kernel: original_w, init_bias: original_b})
    #[...] train the model on your new task

### Freezing the Lower Layers

Imagine you use old DNN picture-classifier model to classify new pictures. The lower layers of DNN learned to detected low-level features in pictures that will be useful also for your actual task. It is generally good idea to freeze weight of pretrained lower layers when training new DNN.

In [ ]:
train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 
                               scope="hidden[34]|outputs") # gets the list of all trainable variables in hidden layers 3,4 
                                                           # and in the output layer
training_op = optimizer.minimize(loss, var_list=train_vars) # proviced list of layers to the optimizer
                                                            #so layers 1 and 2 are frozen

Another option: add a stop_gradient() layer in the graph. Any layer below it will be frozen

In [ ]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1") #reused frozen
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") #reused frozen
    hiden2_stop = tf.stop_gradient(hidden2)
    hidden3 = tf.layers.dense(hidden2_stop, n_hidden3, activation=tf.nn.relu, name="hidden3") #reused not frozen
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4") #new
    logits = tf.layers.dense(n_hidden4, n_outputs, name="outputs") #new

### Caching the Frozen Layers

Since the frozen layers dont change you can use the outputs of the topmost frozen layer for each training instance. You can store(cache) it. It will give you huge speed boost as you will go through frozen layers once per trainin instance. Then feed training operations with these outputs.

In [ ]:
n_batches = mnist.train.num_examples // batch_size

with tf.Session as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")
    
    h2_cache = sess.run(hidden2, feed_dict={X: mnist.train.images})
    
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(mnist.train.num_examples)
        hidden2_batches = np.array_split(h2_cache[shuffled_idx], n_batches)
        y_batches = np.array_split(mnist.train.labels[shuffled_idx], n_batches)
        for hidden2_batch, y_batch in zip(hidden2_batches, y_batches):
            sess.run(training_op, feed_dict={hidden2:hidden2_batch, y:y_batch})
            
        save_path = saver.save(sess, "./my_new_model_final.ckpt")

## Faster Optimizers

To speed up you can use instead of GradientDescentOptimizer:

In [ ]:
momentum_optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate=0.9)
nag_optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate=0.9, use_nesterov=True)
optimizer = tf.train.RMSPopOptimizer(learning_rate=learning_rate, momentum=0.9, decay=0.9, epsilon=1e-10)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

Preffered nag_optimizer

### Learning Rate Scheduling

If you want to speed up your training you can manipulate with learning rate. Start with high value, then decrease it to be get to optimum.

In [ ]:
initial_learnig_rate = 0.1
decay_steps = 10000
decay_rate = 1/10
global_step = tf.Variable(0, trainable=False, name="global_step")
learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, decay_steps, decay_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
training_op = optimizer.minimize(loss, global_step=global_step)

## Avoiding Overfitting Through Regularization

Regularization techniques:

### Early Stopping

You can find it in chapter 4.

With tensorflow you can make it by saving checkpoints at the best performance on the validation set and break training if the new best performance appeared after some limit steps.

### l1 and l2 Regularization

Like we did in Chapter 4 (where??) you can use l1 and l2 regularization to constrain neural network's connection weights (but not biases).

In [ ]:
#[...]
scale = 0.001 #l1 regularization parameter
my_dense_layer = partial(tf.layers.dense, activation = tf.nn.relu, kernel_regularizer = tf.contrib.layers.l1_regularizer(scale))

with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden2, name="hidden2")
    logits = my_dense_layer(hidden2, n_outputs, activation=None, name="outputs")
    
#add regularization losses to overall loss
reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([base_loss] + reg_losses, name="loss") 

### Dropout

This is the most popular regularization technique for dnn. At every trainin step, every neuron has a probability of being temporarily dropped out. But it may be active in the next step. During testing neurons are not dropped.

In [ ]:
#[...]
training = tf.placeholder_with_default(False, shape=(), name="training")
dropout_rate = 0.5 # == 1 - keep_prob
X_drop = tf.layers.dropout(X, dropout_rate, training=training)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X_drop, n_hidden1, activation = tf.nn.relu, name="hidden1")
    hiden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    hidden2 = tf.layers.dense(hidden1_drop, n_hidden2, activation = tf.nn.relu, name="hidden2")
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training = training)
    logits = tf.layers.dense(hidden2_drop, n_outputs, name="outputs")

### Max-Norm Regularization

In [2]:
def max_norm_regularizer(threshold, axes=1, name="max_norm", collection="max_norm"):
    def max_norm(weights):
        clipped = tf.clip_by_norm(weights, clip_norm=threshold, axes=axes)
        clip_weights = tf.assign(weights, clipped, name=name)
        tf.add_to_collection(collection, clip_weights)
        return None
    return max_norm

In [ ]:
max_norm_reg = max_norm_regularizer(threshold = 1.0)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, kernel_regularizer = max_norm_reg, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, kernel_regularizer = max_norm_reg, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    
clip_all_weights = tf.get_collection("max_norm")

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(trainin_op, feed_dict={X: X_batch, y: y_batch})
            sess.run(clip_all_weights)

## Default DNN configuration:

* initialization - HE initialization
* Activation function - ELU
* Normalization - Batch Normalization
* Regularization - Dropout
* Optimizer - Nesterov Accelerated Gradient
* Learning rate schedule - None

## Exercises

In [2]:
import tensorflow as tf
from functools import partial

In [11]:
tf.reset_default_graph()

### Construction

In [12]:
n_inputs = 28*28 #Mnist size of picture
n_hidden1 = 500
n_hidden2 = 500
n_hidden3 = 500
n_hidden4 = 500
n_hidden5 = 500
n_outputs = 5

In [13]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [14]:
he_init = tf.contrib.layers.variance_scaling_initializer()
my_dense_layer = partial(tf.layers.dense, activation = tf.nn.elu, kernel_initializer = he_init)

In [15]:
with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    hidden2 = my_dense_layer(hidden1, n_hidden2, name="hidden2")
    hidden3 = my_dense_layer(hidden2, n_hidden3, name="hidden3")
    hidden4 = my_dense_layer(hidden3, n_hidden4, name="hidden4")
    hidden5 = my_dense_layer(hidden4, n_hidden5, name="hidden5")
    logits = my_dense_layer(hidden5, n_outputs, activation=None, name="outputs")

In [16]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [18]:
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    training_op = optimizer.minimize(loss)

In [21]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

In [22]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

### Execution

In [23]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [24]:
print(mnist)

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001C05F03B710>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001C05F03B828>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000001C05F03B400>)
